# Acquisition and Prep
Goal is to predict the values of single unit properties using the obervations from 2017

1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [1]:
# prepare
import pandas as pd
import numpy as np
import wrangle

# turn off pink warning boxes
import warnings
warnings.filterwarnings("ignore")

# acquire
import env

In [3]:
zillow_df = wrangle.get_zillow_data()
zillow_df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,9.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
2,0.0,0.0,73026.0,1434941.0,1959.0,20800.37,6037.0
3,0.0,0.0,5068.0,1174475.0,1948.0,14557.57,6037.0
4,0.0,0.0,1776.0,440101.0,1947.0,5725.17,6037.0
...,...,...,...,...,...,...,...
2985212,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985213,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2985215,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. Acquisition and Prep - Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [4]:
pd.isnull(zillow_df).sum()

bedroomcnt                       2945
bathroomcnt                      2957
calculatedfinishedsquarefeet    45097
taxvaluedollarcnt               34266
yearbuilt                       47833
taxamount                       22752
fips                             2932
dtype: int64

In [5]:
(zillow_df).shape[0]

2985217

In [6]:
(zillow_df).shape[0] - pd.isnull(zillow_df).sum()

bedroomcnt                      2982272
bathroomcnt                     2982260
calculatedfinishedsquarefeet    2940120
taxvaluedollarcnt               2950951
yearbuilt                       2937384
taxamount                       2962465
fips                            2982285
dtype: int64

In [7]:
# Should be percentage of nulls but lecture had a different answer
(pd.isnull(zillow_df).sum() / (zillow_df).shape[0])*100

bedroomcnt                      0.098653
bathroomcnt                     0.099055
calculatedfinishedsquarefeet    1.510677
taxvaluedollarcnt               1.147856
yearbuilt                       1.602329
taxamount                       0.762156
fips                            0.098217
dtype: float64

In [ ]:
# This is the method Ryan used which has different results, well at least in the lesson, hmm...
zillow_df.isna().mean()

In [ ]:
# percentage of data left if we dropped all rows with any null values; lecture different number
round(zillow_df.dropna().shape[0] / zillow_df.shape[0], 4)

In [ ]:
# drops the rows with any null values and returns a new null-free df
zillow_df = zillow_df.dropna()


In [8]:
# null free df; different from lecture
zillow_df.shape[0]

2985217

In [ ]:
zillow_df.columns.tolist()

In [ ]:
# lists number of results for each of the values in each column; different from lecture
for column in zillow_df.columns:
    print(column)
    print(zillow_df[column].value_counts())
    print("-----------------")

Based on these results we can change bedrooms to an integer since they're all whole numbers, same with calculatedfinishedsquarefeet, taxvaluedollarcnt, and yearbuilt
However the bathrooms include half baths which we'd like to keep so they'll stay as floats
FIPS can also be changed to an integer because this number is categorical representing county zip codes

In [ ]:
# We can use the following code to figure out that 100% of calculatedfinishedsquarefeet can be converted to int w/o data loss
(zillow_df.calculatedfinishedsquarefeet == zillow_df.calculatedfinishedsquarefeet.astype(int)).mean()

In [ ]:
# Same with taxvaluedollarcnt. 100% of taxvaluedollarcnt can lose the deicimal and be OK
(zillow_df.taxvaluedollarcnt == zillow_df.taxvaluedollarcnt.astype(int)).mean()

In [ ]:
# This is not the case for out bathrooms
(zillow_df.bathroomcnt == zillow_df.bathroomcnt.astype(int)).mean()

In [ ]:
# converts our fips, yearbuilt, bedrooms, taxvaluedollarcnt, to integers
zillow_df["fips"] = zillow_df["fips"].astype(int)
zillow_df["yearbuilt"] = zillow_df["yearbuilt"].astype(int)
zillow_df["bedroomcnt"] = zillow_df["bedroomcnt"].astype(int)
zillow_df["taxvaluedollarcnt"] = zillow_df["taxvaluedollarcnt"].astype(int)
zillow_df["calculatedfinishedsquarefeet"] = zillow_df["calculatedfinishedsquarefeet"].astype(int)

In [ ]:
zillow_df.describe().round(1)

In [ ]:
zillow_df.dtypes

In [1]:
# import modules for acquire and prep stages
import pandas as pd
import env
import os
from sklearn.model_selection import train_test_split

The below function will be what we use to access the database holding our Zillow data

In [2]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
    '''
    Function allows user to access Codeup database using their own 
    credentials stored in  their env.py file
    '''
    
# Returns with correct address/password combinat to access the database
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
def new_zillow_data():
    '''
    SQL query that joins the customers table with, contract, payment, and \n
    internet service options
    '''
    
# SQL query that joins three other tables to customer table
    sql_query = '''
                SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, fips FROM properties_2017
                join propertylandusetype using (propertylandusetypeid)
                where propertylandusedesc = "Single Family Residential"
                '''
# Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    
# Returns the called dataframe
    return df

In [4]:
def get_zillow_data():
    '''
    Function allows user to access zillow_data from Codeup database and write it\n
    to a csv file then returns the dataframe.
    '''
    
# if statement that checks if there's already a .csv file to use 
    if os.path.isfile('zillow.csv'):
        
# If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
# Alternative if no csv file found then
    else:
        
# Read fresh data from db into a DataFrame
        df = new_zillow_data()
        
# Cache data for local use
        df.to_csv('zillow.csv')

# Returns requested df
    return df

Since there are relatively few nulls compared to the number of rows of data we have they'll be dropped using the below function

In [5]:
def handle_nulls(df):
    '''
    Gets rid of the rows with any null values and returns a new null-free df
    '''
    
# drops the rows with any null values and returns a new null-free df
    df = df.dropna()
    
    
    return df

We'll clean up the data by converting it into data types that are easier to work with for our purposes

In [6]:
def float_to_int(df):
    '''
    Converts our fips, bedrooms, calculatedfinishedsquarefeet, taxvaluedollarcnt, and yearbuilt from floats to integers
    '''
# converts our fips, bedrooms, calculatedfinishedsquarefeet, taxvaluedollarcnt, and yearbuilt from floats to integers
    df["fips"] = df["fips"].astype(int)
    df["yearbuilt"] = df["yearbuilt"].astype(int)
    df["bedroomcnt"] = df["bedroomcnt"].astype(int)
    df["taxvaluedollarcnt"] = df["taxvaluedollarcnt"].astype(int)
    df["calculatedfinishedsquarefeet"] = df["calculatedfinishedsquarefeet"].astype(int)
    
    return df

In [7]:
def clean_zillow(df):
    '''
    Groups our functions used to clean up our data into a single function for ease of use
    '''
    
    df = handle_nulls(df)
    df = float_to_int(df)
    
    return df

In [8]:
def split_zillow(df):
    '''
    Takes our df and splits it into train, validate, and test dfs for exploration, fitting, validation, and testing
    '''
    
# splits the full data set 60/40 into train and test dataframes stratified 
# around taxvaluedollarcnt, the target variable, using the train_test_split function
    train, test = train_test_split(df, 
                               train_size = 0.75, 
                               stratify = df.fips, 
                               random_state=2468)

# splits the train dataframe 60/40 into the new train and validate dataframes
# they're stratified around taxvaluedollarcnt again using the train_test_split function
    train, validate = train_test_split(train,
                                    train_size = 0.75,
                                    stratify = train.fips,
                                    random_state=2468)
    
# returns the three dataframes we'll use for training, validation, and testing
    return train, validate, test

5. Scaling Numeric Data - Based on the work you've done, choose a scaling method for your dataset. Write a function within your prepare.py (wrangle.py for me) that accepts as input the train, validate, and test data splits, and returns the scaled versions of each. Be sure to only learn the parameters for scaling from your training data!

In [1]:
def scale_zillow(train, validate, test,
                 cols_to_scale = ['bedroomcnt', 'bathroomcnt', 'calculatedfinishedsquarefeet', 'taxvaluedollarcnt']):
    '''
    Accepts train, valide, and test as inputs from split data then returns scaled versions for each one
    '''
    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    
    scaler = MinMaxScaler()
    
    scaler.fit(train[cols_to_scale])
    
    train_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(train[cols_to_scale]), columns=train[cols_to_scale].columns.values).set_index([train.index.values])
                                                  
    validate_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(validate[cols_to_scale]), columns=validate[cols_to_scale].columns.values).set_index([validate.index.values])
    
    test_scaled[cols_to_scale] = pd.DataFrame(scaler.transform(test[cols_to_scale]), columns=test[cols_to_scale].columns.values).set_index([test.index.values])
    
    return train_scaled, validate_scaled, test_scaled

3. Acquisition and Prep - Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow

In [9]:
def wrangle_zillow():
    '''
    Function that acquires zillow data using the new_zillow_data function and 
    caches it, as a csv file, if there isn't already a local copy
    '''
    df = get_zillow_data()
    df = clean_zillow(df)
    train, validate, test = split_zillow(df)
    
    return train, validate, test
